In [1]:
state = 'NE' 
year = 2020

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json
from epsilon_constraint import*
import networkx as nx

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of NE has 3 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from NE_plans_2020 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 0 plans from file.


In [6]:
obj_type = 'inverse_Polsby_Popper'
starting_deviation = 0.01 
verbose = True
contiguity = 'lcut'               # {'lcut', 'scf', 'shir'}
warm_start_mode = 'user'          # {'None', 'user', 'refinement'}
warm_starts = warm_starts         # if you have user define warm starts else it is None
time_limit = 7200                 # Time limit in seconds, adjust as needed
multiplier = 0                    # Adjust multiplier as needed
max_B = True                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True
symmetry_breaking = 'orbitope'    # {None, 'orbitope', 'rsum'}              
sizes = None

In [ ]:
print(f"Running epsilon constraint method for state {state}, objective {obj_type}")
print(f"{'#' * 100}\n")
(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity,        
            cutoff=None,       
            verbose=verbose,
            warm_start_mode = warm_start_mode,
            warm_starts=warm_starts,  
            starting_deviation=starting_deviation, 
            time_limit=time_limit, 
            sizes=sizes,      
            max_B=max_B,      
            symmetry_breaking=symmetry_breaking,
            state=state,
            year=year
        )

Running epsilon constraint method for state NE, objective inverse_Polsby_Popper
####################################################################################################

L = 647297 and U = 660373

****************************************
Trying deviation = 6538.346666666666
****************************************
No valid warm start used.

****************************************
Running labeling model!
****************************************
L = 647297 and U = 660373
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Wi

H16824   960                       2.1358768    2.10817  1.30%  68.9   53s
H18038     2                       2.1340332    2.13403  0.00%  65.3   54s

Cutting planes:
  Gomory: 7
  Cover: 7
  StrongCG: 1
  Flow cover: 1
  RLT: 6
  Lazy constraints: 250

Explored 18040 nodes (1184114 simplex iterations) in 54.33 seconds (25.54 work units)
Thread count was 20 (of 20 available processors)

Solution count 6: 2.13403 2.13588 2.34419 ... 2.70259

Optimal solution found (tolerance 0.00e+00)
Best objective 2.134033168630e+00, best bound 2.134033168630e+00, gap 0.0000%

User-callback calls 37951, time in user-callback 4.24 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
[[9, 16, 26, 46, 76], [6, 7, 10, 21, 28, 47, 61, 65, 69, 71, 87, 91], [0, 1, 2, 3, 4, 5, 8, 11, 12, 13, 14, 15, 17, 18, 19, 20, 22, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 48, 49, 50, 51, 52, 53, 54, 55, 56

     0     0    1.19312    0  390          -    1.19312      -     -    4s
     0     0    1.19312    0  391          -    1.19312      -     -    4s
     0     0    1.19337    0  391          -    1.19337      -     -    4s
     0     2    1.19756    0  391          -    1.19756      -     -    4s
    15    26    1.39987    4  383          -    1.26681      -   153    5s
  1544  1396    1.97704   18  398          -    1.32657      -  84.0   10s
  1552  1402    4.46307   35  397          -    1.32657      -  83.6   15s
  1675  1523    2.08076   19  423          -    1.48114      -  98.7   20s
  3138  2322    3.20885   45  260          -    1.51068      -  98.1   25s
  5755  3758    2.65782   30  496          -    1.66987      -  84.4   30s
  7842  5483    2.90184   31  587          -    1.75578      -  78.4   35s
H 8085  1863                       2.9478069    1.75578  40.4%  78.2   35s
H 8091  1582                       2.8012070    1.75578  37.3%  78.2   36s
H 8241  1290             

In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
if any(round(x[1][0], 2) != round(x[1][1], 2) for x in result):
    min_deviation = max(x[2] for x in result if round(x[1][0], 2) != round(x[1][1], 2))
else:
    min_deviation = min(x[2] for x in result)
    
print('min_deviation:' , min_deviation)

In [ ]:
from enumeration import enumerate_and_solve_k2_subproblems
from metrics import *
import math

epsilon = 1 / (2 * G._k)

while True:
    print(f"\n{'*' * 40}\nTrying deviation = {min_deviation}\n{'*' * 40}")
    L = math.ceil(G._ideal_population - min_deviation)
    U = math.floor(G._ideal_population + min_deviation)
    G._L = L
    G._U = U
    print(f'L = {L} and U = {U}')

    new_plans = enumerate_and_solve_k2_subproblems(G, min_deviation, L, U, G._k, obj_type = obj_type, root='Douglas')
    
    if not new_plans:
        print("No plans found. Reducing deviation or stopping.")
        break  

    enumeration_plans = {}
    for i, plan in enumerate(new_plans):
        obj_value = compute_obj(G, plan, obj_type)
        dev = observed_deviation_persons(G, plan, G._ideal_population, year=year)
        enumeration_plans[i] = [obj_value, dev]

    if obj_type in {"inverse_Polsby_Popper", "cut_edges", "perimeter"}:
        best_obj_value = min(val[0] for val in enumeration_plans.values())
    else:
        best_obj_value = max(val[0] for val in enumeration_plans.values())
    print(f"best_obj_value {obj_type}: {best_obj_value}")

    best_plan_index = None
    best_dev = float('inf')

    for i, (val, dev) in enumeration_plans.items():
        if val == best_obj_value and dev < best_dev:
            best_plan_index = i
            best_dev = dev

    if best_plan_index is not None:
        best_entry = (new_plans[best_plan_index], [best_obj_value, best_obj_value], best_dev)

        # Check if an entry with the same deviation already exists
        existing_index = next((i for i, entry in enumerate(result) if math.isclose(entry[2], best_dev)), None)

        if existing_index is not None:
            result[existing_index] = best_entry
            print(f"Replaced plan at index {existing_index} with new plan {best_plan_index} (dev={best_dev}, {best_obj_value} {obj_type})")
        else:
            result.append(best_entry)
            print(f"Added plan {best_plan_index} with {best_obj_value} {obj_type} and deviation {best_dev} to result")

    if best_dev < epsilon:
        print("Deviation is too small, stopping early.")
        break

    min_deviation = best_dev - epsilon

In [ ]:
# Determine the senses for the Pareto frontier
# Minimize the first objective ('deviation').
# Maximize the second objective for compactness types, otherwise minimize it.
senses = ['min', 'max' if obj_type in ['average_Polsby_Popper','bottleneck_Polsby_Popper'] else 'min']
obj_names = ['deviation_persons', obj_type]
infeasible_region = [0, min(round(r[2],1) for r in result)]

In [ ]:
from pareto import ParetoFrontier

print(f"\n{'#' * 100}\nPareto Frontier for state {state},  objective {obj_type}\n{'#' * 100}\n")
pareto = ParetoFrontier(senses, obj_names, state=state, level='county')

if not  plans:
    print("No plan found!")
else:
    for plan, obj_bound, dev in result:
        upper_bound = [dev,  1/obj_bound[0] if  obj_type == 'bottleneck_Polsby_Popper' else obj_bound[0]]
        lower_bound = [dev,  1/obj_bound[1] if  obj_type == 'bottleneck_Polsby_Popper' else obj_bound[1]]
        
        pareto.add_plan(plan, upper_bound, lower_bound)
            
print("Pareto front plans:", pareto.plans)
print("Pareto front upper bounds:", pareto.upper_bounds)
print("Pareto front lower bounds:", pareto.lower_bounds) 

extra_points = None
extra_colors = None
        
pareto.tighten_lower_bounds()

if infeasible_region[1] > 0.5:
    max_dev = max(upper_bound[0] for upper_bound in pareto.upper_bounds)
    max_obj = max(upper_bound[1] for upper_bound in pareto.upper_bounds)
    min_obj = min(upper_bound[1] for upper_bound in pareto.upper_bounds)
    o1lim=[-100, max_dev+100]
    o2lim=[min_obj*0.9, max_obj*(1.1)]
else:
    o1lim = None
    o2lim = None

pareto.plot_with_custom_x_ranges(method = 'epsilon_constraint_method', splits=None,
                                 o1lim=o1lim, o2lim=o2lim, infeasible_region = infeasible_region,
                                 extra_points=extra_points, extra_colors=extra_colors)

In [ ]:
from draw import *

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: round(x, 0)}

for plan, obj_bound, dev in result:
    ub = format_obj.get(obj_type, lambda x: round(x, 4))(obj_bound[0])
    lb = format_obj.get(obj_type, lambda x: round(x, 4))(obj_bound[1])
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    if ub != lb:
        title = f"{round(dev, 2)}-person deviation ({deviation_percentage}%), obj_bounds [{ub}, {lb}] {obj_type}"
    else:
        title = f"{round(dev, 2)}-person deviation ({deviation_percentage}%), with {ub} {obj_type}"

    draw_plan(filepath, filename2, G, plan, title=title, year=year)